<a href="https://colab.research.google.com/github/adarshlearnngrow/StepUp-AI/blob/Base-commit/AI_Hackthon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving JobsDatasetProcessed.csv to JobsDatasetProcessed (1).csv
User uploaded file "JobsDatasetProcessed (1).csv" with length 13403035 bytes
mv: cannot stat 'kaggle.json': No such file or directory


In [27]:
jobs_desc = pd.read_csv("JobsDatasetProcessed.csv")

In [30]:
jobs_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           3000 non-null   int64 
 1   Query        3000 non-null   object
 2   Job Title    3000 non-null   object
 3   Description  3000 non-null   object
 4   IT Skills    2990 non-null   object
 5   Soft Skills  2978 non-null   object
 6   Education    1563 non-null   object
 7   Experience   1572 non-null   object
 8   Token Usage  3000 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 211.1+ KB


In [45]:
jobs_desc['Query'].unique()

array(['Artificial Intelligence', 'Big Data Engineer', 'Business Analyst',
       'Business Intelligence Analyst', 'Cloud Architect',
       'Cloud Services Developer', 'Data Analyst', 'Data Architect',
       'Data Engineer', 'Data Quality Manager', 'Data Scientist',
       'Data Visualization Expert', 'Data Warehousing',
       'Data and Analytics Manager', 'Database Administrator',
       'Deep Learning', 'Full Stack Developer', 'IT Consultant',
       'IT Systems Administrator', 'Information Security Analyst',
       'Machine Learning', 'Network Architect', 'Statistics',
       'Technical Operations', 'Technology Integration'], dtype=object)

In [47]:


req_jobs_desc = jobs_desc.query("Query in ['Artificial Intelligence', 'Business Analyst', 'Business Intelligence Analyst', 'Data Analyst', 'Machine Learning']")


req_jobs_desc = req_jobs_desc[['Query', 'Description']]

In [50]:
req_jobs_desc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 0 to 2519
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Query        600 non-null    object
 1   Description  600 non-null    object
dtypes: object(2)
memory usage: 14.1+ KB


In [53]:
job_desc_json = req_jobs_desc.to_json(orient="records")

In [10]:
import openai
import json
import time
from openai import OpenAI
from google.colab import userdata
import pandas as pd
import kagglehub
pd.set_option('display.max_colwidth', None)

In [11]:
path = kagglehub.dataset_download("meerawks/it-skills-from-jobs")

print("Path to dataset files:", path)

100%|██████████| 4.04M/4.04M [00:00<00:00, 133MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/meerawks/it-skills-from-jobs/versions/1


In [12]:
path

'/root/.cache/kagglehub/datasets/meerawks/it-skills-from-jobs/versions/1'

In [ ]:
data = {'title': ['Data Scientist'], 'level': ['Data Scientist 1'], 'Description': ["""Primary Responsibilities

Develop Data Models: Build, design, and prepare predictive and prescriptive data science models, tools, and databases using languages such as Python or R
Data Analysis: Conduct and document actionable data analysis and research to identify trends and insights
Forecasting: Prepare forecasts and analyze trends to support strategic decision-making
Reporting: Provide detailed summaries, reports, and recommendations to assist managerial decision-making processes
Collaboration: Communicate effectively with management, colleagues, and stakeholders to present findings and insights
Data Visualization: Create compelling data visualizations to represent complex data in an understandable format

You will be rewarded and recognized for your performance in an environment that will challenge you and give you clear directions on what it takes to succeed in your role, as well as providing development for other roles you may be interested in.

Required Qualifications

Currently pursuing or recently completed a degree in Data Science, Statistics, Computer Science, Mathematics, or a related field
Proficiency with data science tools such as Python, R, SQL, Excel, and data visualization platforms like Tableau or Matplotlib
Exposure to programming languages commonly used in data science (e.g., Python, R)

Soft Skills

Analytical and problem-solving skills with the ability to interpret complex data sets
Verbal and written communication skills to convey insights and collaborate with team members
"""]}

In [ ]:
pd.DataFrame(data)

,title,level,Description
0,Data Scientist,Data Scientist 1,"Primary Responsibilities\n\nDevelop Data Models: Build, design, and prepare predictive and prescriptive data science models, tools, and databases using languages such as Python or R \nData Analysis: Conduct and document actionable data analysis and research to identify trends and insights \nForecasting: Prepare forecasts and analyze trends to support strategic decision-making \nReporting: Provide detailed summaries, reports, and recommendations to assist managerial decision-making processes \nCollaboration: Communicate effectively with management, colleagues, and stakeholders to present findings and insights \nData Visualization: Create compelling data visualizations to represent complex data in an understandable format \n\nYou will be rewarded and recognized for your performance in an environment that will challenge you and give you clear directions on what it takes to succeed in your role, as well as providing development for other roles you may be interested in.\n\nRequired Qualifications\n\nCurrently pursuing or recently completed a degree in Data Science, Statistics, Computer Science, Mathematics, or a related field\nProficiency with data science tools such as Python, R, SQL, Excel, and data visualization platforms like Tableau or Matplotlib\nExposure to programming languages commonly used in data science (e.g., Python, R) \n\nSoft Skills\n\nAnalytical and problem-solving skills with the ability to interpret complex data sets \nVerbal and written communication skills to convey insights and collaborate with team members \n"


In [ ]:
client = OpenAI(api_key=userdata.get("OPEN_AI_KEY"))


In [ ]:
def generate_student_resume(role: str, level: str = "Entry-Level"):
    prompt = f"""
You are generating a realistic student resume targeting the role: {role} ({level}).

Format your output as valid JSON with the following structure:
{{
  "name": "<Full Name>",
  "education": "<Degree, University, Years>",
  "work_experience": [
    {{
      "role": "<Internship or job title>",
      "company": "<Realistic company name (can be a tech startup, research lab, NGO, etc.)>",
      "duration": "<Start–End>",
      "description": "<Brief but realistic description of responsibilities>"
    }},
    {{
      "role": "<Optional second internship or research assistant role>",
      "company": "<Another realistic organization or university department>",
      "duration": "<Start–End>",
      "description": "<Another realistic description>"
    }}
  ],
  "personal_projects": [
    {{
      "title": "<Project 1: realistic, detailed, and unique (e.g. health app, recommendation engine, game, fintech tool)>",
      "description": "<What it does, why it was built, and which tools/languages were used>"
    }},
    {{
      "title": "<Project 2: unrelated to target job is okay>",
      "description": "<Details and tech stack>"
    }}
  ],
  "technical_skills": ["<Realistic tech skills: some aligned, some adjacent>"],
  "soft_skills": ["<Soft skills like teamwork, leadership, communication, etc.>"]
}}

Important guidelines:
- Use **diverse and realistic names** (no more 'John Doe')
- Use **plausible but not copyrighted company names** (e.g. tech startups, labs, NGOs, departments)
- Use **varied and creative projects** — not all should be recommendation systems!
- Projects can be from coursework, hackathons, personal exploration, or clubs
- Leave **some skill gaps** compared to what a real job would require
- Output must be **valid JSON**, no markdown, no explanation, no comments
"""


    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8
    )
    return response.choices[0].message.content.strip()

# Generate resumes and store in a list
resumes = []
for i in range(10):  # Generate 10 student resumes
    try:
        resume_json = generate_student_resume(role="Data Scientist")
        resume_dict = json.loads(resume_json)
        resumes.append(resume_dict)
        print(f"✅ Generated resume {i+1}")
        time.sleep(1)
    except Exception as e:
        print(f"❌ Error at resume {i+1}: {e}")

# Save all to a JSON file
with open("student_resumes.json", "w", encoding="utf-8") as f:
    json.dump(resumes, f, indent=2, ensure_ascii=False)

✅ Generated resume 1
